In [ ]:
import tensorflow as tf
import os
import numpy as np
from model import *
from utility import *

model_name = "WGAN_for_irasutoya"

In [ ]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.g_bn0 = BatchNormalization(name = 'g_bn0')
        self.g_bn1 = BatchNormalization(name = 'g_bn1')
        self.g_bn2 = BatchNormalization(name = 'g_bn2')
        self.g_bn3 = BatchNormalization(name = 'g_bn3')

    def __call__(self, z):
        with tf.variable_scope('g', reuse=self.reuse):

            fc0 = full_connection_layer(z, 512*6*6, name="fc0")
            fc0 = tf.reshape(fc0, [-1, 6, 6, 512])

            batch_size = tf.shape(fc0)[0]
            deconv0 = deconv2d_layer(fc0, [batch_size, 12, 12, 256], kernel_size=5, name="deconv0")
            deconv0 = self.g_bn0(deconv0)
            deconv0 = lrelu(deconv0, leak=0.3)

            deconv1 = deconv2d_layer(deconv0, [batch_size, 24, 24, 128], kernel_size=5, name="deconv1")
            deconv1 = self.g_bn1(deconv1)
            deconv1 = lrelu(deconv1, leak=0.3)

            deconv2 = deconv2d_layer(deconv1, [batch_size, 48, 48, 64], kernel_size=5, name="deconv2")
            deconv2 = self.g_bn2(deconv2)
            deconv2 = lrelu(deconv2, leak=0.3)

            deconv3 = deconv2d_layer(deconv2, [batch_size, 96, 96, 3], kernel_size=5, name="deconv3")

            output = tf.nn.tanh(deconv3)
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return output

In [ ]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.d_bn0 = BatchNormalization(name="d_bn0")
        self.d_bn1 = BatchNormalization(name="d_bn1")
        self.d_bn2 = BatchNormalization(name="d_bn2")
        self.d_bn3 = BatchNormalization(name="d_bn3")
        self.d_bn4 = BatchNormalization(name="d_bn4")
        
    def __call__(self, x):
        with tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(x, [-1, 96, 96, 3])

            conv1 = conv2d_layer(x, 64, kernel_size=5, name="d_conv0")
            conv1 = self.d_bn0(conv1)
            conv1 = lrelu(conv1, leak=0.3)
            
            conv2 = conv2d_layer(conv1, 128, kernel_size=5, name="d_conv1")
            conv2 = self.d_bn1(conv2)
            conv2 = lrelu(conv2, leak=0.3)

            conv3 = conv2d_layer(conv2, 256, kernel_size=5, name="d_conv2")
            conv3 = self.d_bn2(conv3)
            conv3 = lrelu(conv3, leak=0.3)

            conv4 = conv2d_layer(conv3, 512, kernel_size=5, name="d_conv3")
            conv4 = self.d_bn3(conv4)
            conv4 = lrelu(conv4, leak=0.3)
            
            disc = full_connection_layer(conv4, 1, name="disc")

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')

        return disc

In [ ]:
class GAN:
    def __init__(self):
        self.batch_size = 64
        self.img_size = 96
        self.z_size = 100
        
        self.epochs = 50000
        self.epoch_saveMetrics = 1000
        self.epoch_saveSampleImg = 1000
        self.epoch_saveParamter = 5000
        self.losses = {"d_loss":[], "g_loss":[]}

        # unrolled counts
        self.steps = 5

        self.dataset = np.load("irasutoya_face_1813x96x96x3_jpg.npy")
        self.dataset = (self.dataset/255)# - 0.5

        self.X_tr = tf.placeholder(tf.float32, shape=[None, self.img_size, self.img_size, 3])
        self.z = tf.placeholder(tf.float32, [None, self.z_size])
        
        self.g = Generator()
        self.d = Discriminator()
        self.Xg = self.g(self.z)



    def loss(self):
        disc_tr = self.d(self.X_tr)
        disc_gen = self.d(self.Xg)
        
        loss_d = -tf.reduce_mean(disc_tr) + tf.reduce_mean(disc_gen)
        loss_g = -tf.reduce_mean(disc_gen)

        return loss_g, loss_d

    def train(self):
        # Optimizer

        self.L_g, self.L_d = self.loss()

        d_opt = tf.train.RMSPropOptimizer(learning_rate=5e-5)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.RMSPropOptimizer(learning_rate=5e-5)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        self.clip_updates = [w.assign(tf.clip_by_value(w, -0.01, 0.01)) for w in self.d.variables]

        saver = tf.train.Saver()
        
        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list= "0"
            )
        )
                
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())

            # preparing noise vec for test
            bs = 100
            test_z = np.random.uniform(-1, 1, size=[bs, self.z_size])

            for epoch in range(self.epochs):
                for step in range(self.steps):
                    # extract images for training
                    rand_index = np.random.randint(0, self.dataset.shape[0], size=self.batch_size)
                    X_mb = self.dataset[rand_index, :].astype(np.float32)
                    X_mb = np.reshape(X_mb, [-1, 96, 96, 3])
                    z = np.random.uniform(-1, 1, size=[self.batch_size, self.z_size])

                    sess.run([self.clip_updates])

                    # train Discriminator
                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={
                        self.X_tr: X_mb,
                        self.z:z,
                    })
         
                # extract images for training
                rand_index = np.random.randint(0, self.dataset.shape[0], size=self.batch_size)
                X_mb = self.dataset[rand_index, :].astype(np.float32)
                X_mb = np.reshape(X_mb, [-1, 96, 96, 3])
                z = np.random.uniform(-1, 1, size=[self.batch_size, self.z_size])

                # train Generator
                _, g_loss_value = sess.run([g_train_op, self.L_g], feed_dict={
                    self.X_tr: X_mb,
                    self.z:z,
                })

                # append loss value for visualizing
                self.losses["d_loss"].append(np.sum(d_loss_value))
                self.losses["g_loss"].append(np.sum(g_loss_value))
                
                # print epoch
                if epoch % 100 == 0:
                    print('epoch:{0}, d_loss:{1}, g_loss{2} '.format(epoch, d_loss_value, g_loss_value))
                
                # visualize loss
                if epoch % self.epoch_saveMetrics == 0:
                    save_metrics(model_name, self.losses, epoch)

                # visualize generated images during training
                if epoch % self.epoch_saveSampleImg == 0:
                    img = sess.run(self.Xg, feed_dict={self.z: test_z})
                    #img = (img+ 0.5)*1.0
                    save_imgs(model_name, img, name=str(epoch))

                # save model parameters 
                if epoch % self.epoch_saveParamter == 0:
                    dir_path = "model_" + model_name
                    if not os.path.isdir(dir_path):
                        os.makedirs(dir_path)

                    saver.save(sess, dir_path + "/" + str(epoch) + ".ckpt")

In [ ]:
gan = GAN()
gan.train()